In [1]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration
import torch

processor = AutoProcessor.from_pretrained("facebook/musicgen-stereo-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-stereo-small")

inputs = processor(
    text=["80s pop track with bassy drums and synth"],
    padding=True,
    return_tensors="pt",
)

res = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=256)

/home/bas/anaconda3/envs/musiclm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/bas/anaconda3/envs/musiclm/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/bas/anaconda3/envs/musiclm/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), per

tensor([[ 2775,     7,  2783,  1463,    28,  7981,    63,  5253,     7,    11,
         13353,     1]])


KeyboardInterrupt: 

In [2]:
from IPython.display import Audio

sampling_rate = model.config.audio_encoder.sampling_rate
Audio(res[0].numpy(), rate=sampling_rate)

In [17]:
res[0].T

tensor([[-0.0644,  0.0134],
        [-0.0648,  0.0166],
        [-0.0497,  0.0523],
        ...,
        [-0.0039, -0.0058],
        [-0.0094, -0.0057],
        [-0.0057, -0.0081]])

In [18]:
import scipy

sampling_rate = model.config.audio_encoder.sampling_rate
scipy.io.wavfile.write("musicgen_out.wav", rate=sampling_rate, data=res[0].numpy().T)

In [ ]:
model.generation_config.temperature, model.generation_config.top_k, model.generation_config.top_p, model.generation_config.min_p, model.generation_config.typical_p, model.generation_config.epsilon_cutoff, model.generation_config.eta_cutoff, model.generation_config.renormalize_logits

In [ ]:
model.generation_config

In [ ]:
inputs = processor(
    text=["80s pop track with bassy drums and synth asd asd sad asd sdaa sd"],
    padding=True,
    return_tensors="pt",
)

In [ ]:
model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=256)

In [ ]:
model = torch.load('./musicgen-stereo-small/model.pt')

In [ ]:
dummy_input_ids = torch.randint(0, 100, ( 20, 512), dtype=torch.int64)
dummy_attention_mask = torch.randint(0, 100, ( 20, 512), dtype=torch.int64)
dummy_input_ids = inputs['input_ids']
dummy_attention_mask = inputs['attention_mask']
dummy_max_length = torch.tensor(256, dtype=torch.int64)
dummy_num_codebooks = torch.tensor(8, dtype=torch.int64)
dummy_decoder_start_token_id = torch.tensor(2048, dtype=torch.int64)

In [ ]:
model(
    dummy_input_ids,
    dummy_attention_mask,
    dummy_max_length,
    dummy_num_codebooks,
    dummy_decoder_start_token_id
)[1][0][0].size()

In [1]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration
from naked_model_export import NakedMusicGen, NakedDecoder
import torch

folder = 'musicgen-stereo-small'
processor = AutoProcessor.from_pretrained("facebook/musicgen-stereo-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-stereo-small")

# Export their config files:
processor.tokenizer.save_pretrained(f'{folder}')
processor.save_pretrained(f'{folder}')
model.config.to_json_file(f'{folder}/config.json')
model.generation_config.to_json_file(f'{folder}/generation_config.json')

naked_decoder = NakedDecoder(model.config.to_dict()['decoder'], model.config.decoder._attn_implementation)
naked_decoder.load_state_dict({k.replace('model.', '').replace('decoder.', ''):v for k,v in model.decoder.state_dict().items()}) # Load the weights of the actual model
naked_decoder.eval()

naked_model = NakedMusicGen(
    model.text_encoder, 
    model.audio_encoder,
    model.enc_to_dec_proj, 
    naked_decoder
)

naked_model.eval()
device = 'cpu'#'cuda:0'
model.to(device)

max_length = 256 #This will be variable later
outputs = processor.tokenizer(["80s pop track with bassy drums and synth"])
input_ids, attention_mask = torch.tensor(outputs['input_ids']), torch.tensor(outputs['attention_mask'])

sampling_rate = model.config.audio_encoder.sampling_rate

del model
del processor

/home/bas/anaconda3/envs/musiclm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/bas/anaconda3/envs/musiclm/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/bas/anaconda3/envs/musiclm/lib/python3.10/site-packages/transformers/models/encodec/modeling_encodec.py:120: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), per

In [2]:
with torch.no_grad():
    output = naked_model(
        input_ids.to(device),
        attention_mask.to(device),
        torch.tensor(256, dtype=torch.int64).to(device),
        torch.tensor(8, dtype=torch.int64).to(device),
        torch.tensor(2048, dtype=torch.int64).to(device)
    )

IndexError: index out of range in self

In [7]:
output.min()

tensor(-0.1451, grad_fn=<MinBackward1>)

In [8]:
output.max()

tensor(0.1730, grad_fn=<MaxBackward1>)

In [10]:
from IPython.display import Audio

Audio(output[0].detach().numpy(), rate=36000)